<div style="background-color:#e6f2ff; padding:20px; border-radius:10px;">
<img style="float:left; margin-right:28px; border-radius:8px; box-shadow:0 2px 8px #00004733;" src='alinco.png' width="120"/>
<div style="margin-left:150px;">
<h1 style="color:#000047; font-size:2.3em; margin-bottom:0;">Actividad 3</h1>
<h2 style="color:#003366; font-size:1.3em; margin-top:0;">Tratamiento de datos Faltantes Práctica</h2>
</div>
<br style="clear:both"/>
</div>

<div style="border-left:6px solid #000047; padding:18px; margin-top:18px; background:#f5f5f5; border-radius:8px;">
<span style="font-size:1.1em;"><b>Objetivo:</b>
Practicar los métodos vistos en clase para el tratamiento de datos faltantes.</span>
<ul style="margin-top:10px;">
<li>Procesamiento de datos nulos.</li>
<li>Identificar y tratar valores faltantes.</li>
</ul>
</div>


In [1]:
from google.colab import drive
import sys
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

drive.mount('/content/drive')


ruta_drive = '/content/drive/MyDrive/HyIA/Tarea 5'

sys.path.append(ruta_drive)
try:
    import HyAIA as hya
    print("Librería HyAIA importada correctamente.")
except ImportError:
    print("No se encontró HyAIA.py en la ruta. Creando versión temporal...")
    codigo_hyaia = """
import pandas as pd
class HyAIA:
    def __init__(self, df):
        self.data = df
        self.df_dqr = self.get_dqr()
    def get_dqr(self):
        columns = pd.DataFrame(list(self.data.columns.values), columns=['Columns_Names'], index=list(self.data.columns.values))
        data_dtypes = pd.DataFrame(self.data.dtypes, columns=['Dtypes'])
        present_values = pd.DataFrame(self.data.count(), columns=['Present_values'])
        missing_values = pd.DataFrame(self.data.isnull().sum(), columns=['Missing_values'])
        unique_values = pd.DataFrame(columns=['Unique_values'])
        for col in list(self.data.columns.values):
            unique_values.loc[col] = [self.data[col].nunique()]
        return columns.join(data_dtypes).join(present_values).join(missing_values).join(unique_values)
"""
    with open('HyAIA.py', 'w') as f:
        f.write(codigo_hyaia)
    import HyAIA as hya
    print(" Librería HyAIA temporal creada e importada.")

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (10, 6)

Mounted at /content/drive
Librería HyAIA importada correctamente.


<div style="border-left:6px solid #003366; border-radius:8px; padding:16px; margin-bottom:16px;">
<h2 style="color:#003366; margin-top:0;">Ejercicio 1</h2>

</div>


El dataset adult.csv fue extraído por Barry Becke de la base de datos de Censos de  1994. Contiene tanto variables numéricas como categóricas. La información de las variables y su contenido se presentan a continuación:

**age:** continuo

**workclass:** Private, Self-emp-not-inc, Self-emp-inc, Federal-gov, Local-gov, State-gov, Without-pay, Never-worked.

**fnlwgt:** continuo.

**education:** Bachelors, Some-college, 11th, HS-grad, Prof-school, Assoc-acdm, Assoc-voc, 9th, 7th-8th, 12th, Masters, 1st-4th, 10th, Doctorate, 5th-6th, Preschool.

**education-num:** continuo.

**marital-status:** Married-civ-spouse, Divorced, Never-married, Separated, Widowed, Married-spouse-absent, Married-AF-spouse.

**occupation:** Tech-support, Craft-repair, Other-service, Sales, Exec-managerial, Prof-specialty, Handlers-cleaners, Machine-op-inspct, Adm-clerical, Farming-fishing, Transport-moving, Priv-house-serv, Protective-serv, Armed-Forces.

**relationship:** Wife, Own-child, Husband, Not-in-family, Other-relative, Unmarried.

**race:** White, Asian-Pac-Islander, Amer-Indian-Eskimo, Other, Black.

**sex:** Female, Male.

**capital-gain:** continuo.

**capital-loss:** continuo.

**hours-per-week:** continuo.

**native-country:** United-States, Cambodia, England, Puerto-Rico, Canada, Germany, Outlying-US(Guam-USVI-etc), India, Japan, Greece, South, China, Cuba, Iran, Honduras, Philippines, Italy, Poland, Jamaica, Vietnam, Mexico, Portugal, Ireland, France, Dominican-Republic, Laos, Ecuador, Taiwan, Haiti, Columbia, Hungary, Guatemala, Nicaragua, Scotland, Thailand, Yugoslavia, El-Salvador, Trinadad&Tobago, Peru, Hong, Holand-Netherlands.

In [5]:
from sklearn.impute import SimpleImputer

cols_adult = ['age', 'workclass', 'fnlwgt', 'education', 'education-num',
              'marital-status', 'occupation', 'relationship', 'race', 'sex',
              'capital-gain', 'capital-loss', 'hours-per-week', 'native-country', 'income']

path_adult = os.path.join(ruta_drive, 'adult.csv')

try:
    if os.path.exists(path_adult):
        df = pd.read_csv(path_adult, names=cols_adult, na_values=['?', ' ?'])
        print(f" Dataset cargado desde Drive: {df.shape}")
    else:
        print(" Archivo local no encontrado. Descargando desde UCI Repository...")
        url_adult = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
        df = pd.read_csv(url_adult, names=cols_adult, na_values=['?', ' ?'])
        print(f" Dataset descargado: {df.shape}")
except Exception as e:
    print(f"Error cargando datos: {e}")

print("\n--- 1.1 Valores Faltantes por Columna ---")
missing_counts = df.isnull().sum()
print(missing_counts[missing_counts > 0])

print("\n--- 1.2 Porcentaje de Información Faltante ---")
missing_percent = (df.isnull().sum() / len(df)) * 100
print(missing_percent[missing_percent > 0].map('{:.2f}%'.format))

df_copy = df.copy()
df_filtered = df_copy.dropna()
print(f"\n--- 1.3 Dimensiones tras dropna: {df_filtered.shape} (Original: {df.shape}) ---")

df_filtered.reset_index(drop=True, inplace=True)
print("--- 1.4 Índices reseteados exitosamente. ---")
print(df_filtered.tail(3))

print("\n--- 1.5 Eliminando nulos en 'native-country' ---")
filas_antes = len(df_copy)
df_copy.dropna(subset=['native-country'], inplace=True)
print(f"Filas eliminadas: {filas_antes - len(df_copy)}")

subset_test = df.iloc[25:31].copy()
subset_test.iloc[1, 0] = np.nan

print("\n--- 1.6 Ejemplo de ffill (Forward Fill) ---")
subset_ffill = subset_test.fillna(method='ffill')
display(subset_ffill)

print("\n--- Ejemplo de bfill (Backward Fill) ---")
subset_bfill = subset_test.fillna(method='bfill')
display(subset_bfill)
print("\n--- 1.7 Imputación 'most_frequent' con Sklearn ---")
imputer = SimpleImputer(strategy='most_frequent')
cols_missing = ['workclass', 'occupation']
df_imputed = df.copy()
df_imputed[cols_missing] = imputer.fit_transform(df[cols_missing])

print("Valores nulos después de imputar (debería ser 0):")
print(df_imputed[cols_missing].isnull().sum())

 Dataset cargado desde Drive: (48843, 15)

--- 1.1 Valores Faltantes por Columna ---
workclass         2799
occupation        2809
native-country     857
dtype: int64

--- 1.2 Porcentaje de Información Faltante ---
workclass         5.73%
occupation        5.75%
native-country    1.75%
dtype: object

--- 1.3 Dimensiones tras dropna: (45223, 15) (Original: (48843, 15)) ---
--- 1.4 Índices reseteados exitosamente. ---
      age     workclass  fnlwgt  education education-num      marital-status  \
45220   2       Private  374983  Bachelors            13  Married-civ-spouse   
45221   2       Private   83891  Bachelors            13            Divorced   
45222   1  Self-emp-inc  182148  Bachelors            13  Married-civ-spouse   

            occupation relationship                race   sex capital-gain  \
45220   Prof-specialty      Husband               White  Male            0   
45221     Adm-clerical    Own-child  Asian-Pac-Islander  Male            2   
45222  Exec-managerial   

/tmp/ipython-input-2125150514.py:46: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset_ffill = subset_test.fillna(method='ffill')


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
25,4,Private,109015,HS-grad,9,Divorced,Tech-support,Unmarried,White,Female,0,0,2,United-States,<=50K
26,4,Local-gov,216851,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,2,United-States,>50K
27,0,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,2,United-States,<=50K
28,3,Private,180211,Some-college,10,Married-civ-spouse,Craft-repair,Husband,Asian-Pac-Islander,Male,0,0,3,South,>50K
29,2,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,4,United-States,<=50K
30,3,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,2,United-States,<=50K



--- Ejemplo de bfill (Backward Fill) ---


/tmp/ipython-input-2125150514.py:50: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  subset_bfill = subset_test.fillna(method='bfill')


,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
25,4,Private,109015,HS-grad,9,Divorced,Tech-support,Unmarried,White,Female,0,0,2,United-States,<=50K
26,0,Local-gov,216851,Bachelors,13,Married-civ-spouse,Tech-support,Husband,White,Male,0,0,2,United-States,>50K
27,0,Private,168294,HS-grad,9,Never-married,Craft-repair,Own-child,White,Male,0,0,2,United-States,<=50K
28,3,Private,180211,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,Asian-Pac-Islander,Male,0,0,3,South,>50K
29,2,Private,367260,HS-grad,9,Divorced,Exec-managerial,Not-in-family,White,Male,0,0,4,United-States,<=50K
30,3,Private,193366,HS-grad,9,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,2,United-States,<=50K



--- 1.7 Imputación 'most_frequent' con Sklearn ---
Valores nulos después de imputar (debería ser 0):
workclass     0
occupation    0
dtype: int64


**1.1- Encontrar el número de valores faltantes por columna**

**1.2.- Encontrar el porcentaje de información faltante**

In [ ]:
# Porcentaje de información faltante


#### Ignorar fila de datos

**1.3.- El comando `dropna()` permite eliminar las filas y/o columnas en las que hayan datos faltantes. Cree una copia del dataset (`df_copy`) y elimine las filas y/o columnas de `df_copy` y el dataframe resultante guardelo en `df_filtered`.**

**1.4.- Note que los índices no cambian. Lo que realiza es la eliminación de la fila (por ejemplo la 14 no está), pero mantiene la indexación. Por tanto, estos no coinciden con el número total de filas. Para reasignar los índices se puede hacer uso del comando `reset_index(drop=True, inplace=True)`. Realice un reseteo del índice de df_filtered.**

**1.5.- Se puede usar el argumento `subset` en el método dropna para seleccionar solo las columnas sobre las que se desea analizar si hay valores nulos para eliminar las filas. Elimine los datos nulos de `df_copy` para la columna: native-country**

### Llenado con valores vecinos

**1.6.-El comando `fillna()` tiene el argumento `method` que permite hacer un llenado hacia atrás ('bfill') o hacia adelante ('ffill'). Utilice `ffill` y `bfill` con el método `fillna` para el dataframe `df_copy` de los índices 25 hasta 30. Tenga en cuenta que al usar ésta opción si el dato con el que se intenta hacer el llenado también es un NaN, éste permanece.**

### Llenado de datos usando sklearn



**1.7.- Es posible también usar el metodo 'SimpleImputer' de la libreria sklearn para hacer llenado, éste nos permite definir cual estrategia usar para el llenado de los datos (media, mediana o moda). importe la clase `SimpleImputer` de `sklearn.impute`. Aplique el llenado de los datos nulos mediante la estrategia 'most_frequent'**

<div style="border-left:6px solid #003366; border-radius:8px; padding:16px; margin-bottom:16px;">
<h2 style="color:#003366; margin-top:0;">Ejercicio 2</h2>

</div>

Trataremos un ejemplo de una base de datos de una tabla de características de marcas de carros.

In [ ]:
archivo = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/auto.csv"

In [ ]:
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

##### Carga de datos desde el url

Para esto usamos el método `read_csv()`

In [ ]:
df = pd.read_csv(archivo, names = headers)

**2.1.- Inspección del archivo. Recordemos que podemos usar el método head().**

In [6]:

url_auto = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DA0101EN/auto.csv"
headers = ["symboling","normalized-losses","make","fuel-type","aspiration", "num-of-doors","body-style",
         "drive-wheels","engine-location","wheel-base", "length","width","height","curb-weight","engine-type",
         "num-of-cylinders", "engine-size","fuel-system","bore","stroke","compression-ratio","horsepower",
         "peak-rpm","city-mpg","highway-mpg","price"]

# 2.1 Carga e Inspección
df_auto = pd.read_csv(url_auto, names=headers)
print("\n--- 2.1 Inspección del Dataset Auto ---")
display(df_auto.head())

# 2.2 Reemplazo de '?' por NaN
print("\n--- 2.2 Reemplazando '?' por NaN ---")
df_auto.replace("?", np.nan, inplace=True)
print("Reemplazo completado.")

# 2.3 Identificación y conteo usando tu librería HyAIA
print("\n--- 2.3 Reporte de Datos Faltantes (HyAIA) ---")
analisis_auto = hya.HyAIA(df_auto)
# Filtramos el reporte para mostrar solo las que tienen faltantes
dqr = analisis_auto.df_dqr
display(dqr[dqr['Missing_values'] > 0])

# ==========================================
# 2.4 y 2.5: Tratamiento (Imputación)
# ==========================================
print("\n--- 2.4 y 2.5 Tratamiento de Faltantes ---")

# A) Reemplazo por la MEDIA (Columnas numéricas)
cols_mean = ["normalized-losses", "stroke", "bore", "horsepower", "peak-rpm"]
for col in cols_mean:
    # Convertimos a float primero, ya que los '?' hacían que fuera objeto
    df_auto[col] = df_auto[col].astype("float")
    avg = df_auto[col].mean(axis=0)
    df_auto[col].replace(np.nan, avg, inplace=True)
    print(f"Columna '{col}': Imputada con media {avg:.2f}")

# B) Reemplazo por la MODA (Categorías)
# "num-of-doors"
mode_doors = df_auto["num-of-doors"].mode()[0]
df_auto["num-of-doors"].replace(np.nan, mode_doors, inplace=True)
print(f"Columna 'num-of-doors': Imputada con moda '{mode_doors}'")

# C) Eliminación de fila entera (Price)
before_drop = df_auto.shape[0]
df_auto.dropna(subset=["price"], axis=0, inplace=True)
df_auto.reset_index(drop=True, inplace=True)
after_drop = df_auto.shape[0]
print(f"Columna 'price': Filas eliminadas {before_drop - after_drop}")

# ==========================================
# 2.6 y 2.7: Corrección de Tipos de Datos
# ==========================================
print("\n--- 2.6 Tipos de Dato Actuales ---")
print(df_auto.dtypes)

print("\n--- 2.7 Conversión de Tipos (Corrección) ---")
# Convertimos price a float (ya que eliminamos los nulos)
df_auto[["price"]] = df_auto[["price"]].astype("float")
df_auto[["normalized-losses"]] = df_auto[["normalized-losses"]].astype("int")

# Verificación final
print("Tipos corregidos:")
print(df_auto[["price", "normalized-losses", "bore", "stroke"]].dtypes)



--- 2.1 Inspección del Dataset Auto ---


,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450



--- 2.2 Reemplazando '?' por NaN ---
Reemplazo completado.

--- 2.3 Reporte de Datos Faltantes (HyAIA) ---


,Columns_Names,Dtypes,Present_values,Missing_values,Unique_values,Max_values,Min_values
normalized-losses,normalized-losses,object,164,41,51,N/A,N/A
num-of-doors,num-of-doors,object,203,2,2,N/A,N/A
bore,bore,object,201,4,38,N/A,N/A
stroke,stroke,object,201,4,36,N/A,N/A
horsepower,horsepower,object,203,2,59,N/A,N/A
peak-rpm,peak-rpm,object,203,2,23,N/A,N/A
price,price,object,201,4,186,N/A,N/A



--- 2.4 y 2.5 Tratamiento de Faltantes ---
Columna 'normalized-losses': Imputada con media 122.00
Columna 'stroke': Imputada con media 3.26
Columna 'bore': Imputada con media 3.33
Columna 'horsepower': Imputada con media 104.26
Columna 'peak-rpm': Imputada con media 5125.37
Columna 'num-of-doors': Imputada con moda 'four'
Columna 'price': Filas eliminadas 4

--- 2.6 Tipos de Dato Actuales ---
symboling              int64
normalized-losses    float64
make                  object
fuel-type             object
aspiration            object
num-of-doors          object
body-style            object
drive-wheels          object
engine-location       object
wheel-base           float64
length               float64
width                float64
height               float64
curb-weight            int64
engine-type           object
num-of-cylinders      object
engine-size            int64
fuel-system           object
bore                 float64
stroke               float64
compression-ratio    fl

/tmp/ipython-input-4234516631.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_auto[col].replace(np.nan, avg, inplace=True)
/tmp/ipython-input-4234516631.py:35: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try u

**Podemos observar que muchas de las entradas están etiquetadas con el símbolo `?`. Esto corresponde datos faltantes que pueden dificultar un análisis posterior.**

Los pasos que debemos seguir a continuación son:

> 1.   Identificar los valores faltantes
> 2.   Tratar los valores faltantes.
> 3.   Corregir el formato de los datos.

Usa tu librería para obtener esta información.

#### Identificación de valores faltantes y tratamiento.


**2.2.-Conversión de `?` a `NaN`. Quienes llenaron esta tabla, identificaron los valores faltantes como `?`.  Por razones de tiempo de computo y conveniencia, replazaremos estos signos de interrogación por `NaN`, para lo cual debemos importantar antes la librería numéricas de python numpy. Realice el reemplazo del caracter `?` por np.nan.**

**Hint: puede hacer uso de la librería de clase**

**2.3.- Identificación y conteo de valores faltantes. Aplique el método `isnull()` y utilice la función de agregación `sum()` guarde este onbjeto tipo serie en la variable `datos_faltantes`.- Comente cuales son las columnas con el número y porcentaje de datos faltantes**

#### Tratamiento de los datos faltantes

Solo debemos eliminar una columna si la mayoria de entradas son vacias. En nuestro caso, ninguna de las columnas cumplen con este criterio como para ser eliminadas.

En este caso tenemos cierta libertad para aplicar diferences métodos de llenado de datos faltantes; no obstante, algunos métodos pueden resultar más adecuados que otros. Aplique los siguientes métodos para cada columna:



**2.4.- Reemplazo por la media:**
<ul>
    <li>"normalized-losses"</li>
    <li>"stroke"</li>
    <li>"bore"</li>
    <li>"horsepower"</li>
    <li>"peak-rpm"</li>
</ul>

**2.5.- Replace por la moda:**
<ul>
    <li>"num-of-doors"
        <ul>
            <li>Razón: 84% de los sedan son 4 puertas. Dado que los carros cuatro puertas son los más frecuentes, la probabilidad de ocurrencia es mayor.</li>
        </ul>
    </li>
</ul>

**2.5.- Eliminación de una fila entera:**
<ul>
    <li>"price": 4 datos faltantes
        <ul>
            <li>Razón: Como, eventualmente, el precio es algo que se puede predecir, ninguna entrada sin precio puede ser usada para una predicción; por ende, cualquier fila sin precio no será útil.</li>
        </ul>
    </li>
</ul>

#### Corrección de datos

**El último paso en el preprocesado de los datos consiste en revisar de que todos los datos estén en el formato adecuado(int, float, text u otro).**

**Podemos usar en Pandas**
<p><b>.dtypes()</b>  para ver el tipo</p>
<p><b>.astype()</b> para cambiar el tipo</p>

**2.6.- obtenga la lista de tipos de dato del dataframe**

**2.7.- Se puede observa que algunas columnas no tienen el tipo correcto. Las variables numéricas deben ser de tipo 'float' o 'int', y las variables con caracteres, como categorias, deben ser del tipo 'object'. Por ejemplo, 'bore' (diámetro del cilindro) y 'stroke' (ciclos/tiempos) son variables numéricas que describen el motor, esperamos entonces que sean de tipo 'float' o 'int'; no obstante, son de tipo 'object'. convierta los tipos de datos a un tipo adecuado usantdo el método "astype()".**
